In [ ]:
import pickle
from operator import itemgetter
import pandas as pd
import plotly.graph_objects as go

from hydra.plotly_charts import graph, supersample_data, add_trace
from hydra.utils import split_dupes

filename = "../2020-05-01 1d count=365 overlap='18h' detrend=True.enviro"
file = None
with open(filename, "rb") as handle:
    file = pickle.load(handle)
interesting, charts, aggregate = itemgetter('data', 'charts', 'aggregate')(file)


pair = "XBTUSD"
startDate = charts[0]['startDate']
endDate = charts[-1]['endDate']
# startDate = "2017-05-15 00:00"
# endDate = "2020-05-16 00:00"


trendline = pd.concat([chart['trendline'] for chart in charts])
trendlines = split_dupes(trendline)
figure = graph(pair, startDate, endDate)

for trend in trendlines[:1]:
    supersampled_trend = supersample_data(trend, 1)
    add_trace(
        figure, 
        go.Scatter(x=supersampled_trend[1440].index, y=supersampled_trend[1440].values), 
        supersampled_trend, 
        {"x": "index", "y": "values"},
        startDate,
        endDate
    )

widget
